In [1]:
import bisect
from collections import defaultdict
import re
import typing

# Some utils from https://github.com/mcpower/adventofcode

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s: str) -> typing.List[int]:
    return lmap(int, re.findall(r"-?\d+", s))

In [2]:
with open("inputs/05.txt", 'r') as fh:
    lines = [line.strip() for line in fh.readlines() if line.strip()]

In [3]:
seeds = ints(lines[0])

maps = defaultdict(list)
next_key = {}
reg_line = re.compile(r"(?P<src>\w+)-to-(?P<dst>\w+) map:")
for line in lines[1:]:
    m = re.match(reg_line, line)
    if m:
        src = m.group("src")
        dst = m.group("dst")
        next_key[src] = dst
    else:
        maps[src].append(tuple(map(int, line.split(" "))))

for k in maps:
    maps[k].sort(key=lambda x: x[1])

In [4]:
# Part 1
min_loc = None
for seed in seeds:
    key = "seed"
    val = seed
    while key != "location":
        i = bisect.bisect_right(list(map(lambda x: x[1], maps[key])), val) - 1
        if i >= 0:
            dst_start, src_start, range_len = maps[key][i]
            if val < src_start + range_len:
                val += dst_start - src_start
        key = next_key[key]
    if min_loc is None or val < min_loc:
        min_loc = val
print(min_loc)

278755257


In [5]:
# Part 2
def chase(key, val_start, val_end):
    if key == "location":
        return val_start
    i = bisect.bisect_right(list(map(lambda x: x[1], maps[key])), val_start) - 1
    if i >= 0:
        dst_start, src_start, range_len = maps[key][i]
        if val_start < src_start + range_len:
            if val_end < src_start + range_len:
                return chase(next_key[key], val_start + dst_start - src_start,
                             val_end + dst_start - src_start)
            else:
                return min(chase(next_key[key], val_start + dst_start - src_start,
                             dst_start + range_len - 1),
                           chase(key, src_start + range_len, val_end))
    return chase(next_key[key], val_start, val_end)

min_loc = None
for i in range(len(seeds) // 2):
    seed_start, seed_len = seeds[2*i], seeds[2*i + 1]
    val = chase("seed", seed_start, seed_start + seed_len - 1)
    if min_loc is None or val < min_loc:
        min_loc = val
print(min_loc)


26829166
